# Obtaining NYC 311 Data

New York's Open Data Portal (https://opendata.cityofnewyork.us/) uses the Socrata Open Data API to give API access to data hosted on the site.

This is significant because the datasets on NYC Open Data are often many millions of rows -- prohibitively large for some.  It's helpful to be able to download only the first, say, 50 thousand rows to get a taste of what the entire dataset is.  We can also specify only certain data, using column names and conditions.

## 311 Overview

In the city of New York, citizens with non-emergency complaints (e.g. trash non-collection, rodent infestations) can call 311 to make a Service Request. These are recorded and shared on New York's open data site at https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9.

## High-Level Data Description

The data dates from 2010 to the current day, with data being updated on a daily basis. At the time of this writing, there are over 20 million rows, each row representing a single service request, and over 40 columns which represent aspects of each service request, such as the street address being referenced, the type of complaint, the agency responsible, the date of the service request, etc.

## Bring in Data via pandas

We're only going to bring in only the rows that have 'Pothole' in the descriptor field.  We'll set an upper limit of 1 million rows.

In [1]:
import pandas as pd
potholes = pd.read_csv("https://data.cityofnewyork.us/resource/fhrw-4uyv.csv?descriptor=Pothole&$limit=1000000")

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


How large is this data?

In [2]:
potholes.shape

(569142, 45)

Let's take a peek at the data in several ways.  We'll start by looking at the first few rows.  We'll scroll to the right to see all the columns.

In [3]:
potholes.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location_city,location,location_address,location_zip,location_state
0,42035216,2019-03-23T15:37:36.000,2019-03-27T07:34:00.000,DOT,Department of Transportation,Street Condition,Pothole,NaN,11355.0,BOOTH MEMORIAL AVENUE,...,NaN,NaN,NaN,40.744876,-73.820516,NaN,POINT (-73.820515773594 40.744876037242),NaN,NaN,NaN
1,42035218,2019-03-23T17:09:03.000,2019-03-27T10:13:00.000,DOT,Department of Transportation,Street Condition,Pothole,NaN,10314.0,NaN,...,NaN,NaN,NaN,40.613078,-74.122557,NaN,POINT (-74.122556992381 40.613077936948),NaN,NaN,NaN
2,42036611,2019-03-23T19:04:12.000,NaN,DOT,Department of Transportation,Street Condition,Pothole,NaN,11377.0,NaN,...,NaN,NaN,NaN,40.740335,-73.898720,NaN,POINT (-73.898720018464 40.740335470005),NaN,NaN,NaN
3,42036612,2019-03-23T09:25:59.000,2019-03-27T13:20:00.000,DOT,Department of Transportation,Street Condition,Pothole,NaN,10314.0,NaN,...,NaN,NaN,NaN,40.614868,-74.139287,NaN,POINT (-74.139287300499 40.614867702584),NaN,NaN,NaN
4,42036613,2019-03-23T13:40:15.000,2019-03-28T09:44:00.000,DOT,Department of Transportation,Street Condition,Pothole,NaN,10475.0,3285 ROMBOUTS AVENUE,...,NaN,NaN,NaN,40.880089,-73.827604,NaN,POINT (-73.827603802742 40.880089328815),NaN,NaN,NaN


Let's look at the overall number of present vs absent values in each column, as well as the column type:

In [4]:
potholes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569142 entries, 0 to 569141
Data columns (total 45 columns):
unique_key                        569142 non-null int64
created_date                      569142 non-null object
closed_date                       566287 non-null object
agency                            569142 non-null object
agency_name                       569142 non-null object
complaint_type                    569142 non-null object
descriptor                        569142 non-null object
location_type                     1900 non-null object
incident_zip                      525046 non-null float64
incident_address                  356993 non-null object
street_name                       356993 non-null object
cross_street_1                    462612 non-null object
cross_street_2                    462547 non-null object
intersection_street_1             202959 non-null object
intersection_street_2             202957 non-null object
address_type                      55

## Cleaning and Preparing Data

We see multiple columns with few to no values, and we also see columns that have data types that aren't quite right (date stamps as string objects).  We'll take that on in this section.

### Dates

Let's begin by converting dates:

In [5]:
for col in ['created_date', 'closed_date', 'due_date', 'resolution_action_updated_date']:
    potholes[col] = pd.to_datetime(potholes[col])

And now let's peek at the dates:

In [6]:
potholes[['created_date', 'closed_date', 'due_date', 'resolution_action_updated_date']].describe()

,created_date,closed_date,due_date,resolution_action_updated_date
count,569142,566287,1872,568949
unique,545013,407126,1575,410211
top,2010-03-26 07:00:00,2011-02-10 00:00:00,2014-06-25 18:44:38,2014-02-23 00:00:00
freq,94,70,17,48
first,2010-01-01 01:57:23,2010-01-01 06:08:33,2010-06-22 16:28:07,2010-01-01 06:08:33
last,2019-07-30 23:14:06,2019-07-30 22:41:04,2019-11-24 09:43:36,2019-07-30 23:14:06


Wonderful, we don't have any outlier dates.  All the dates fall within an expected range of 2010-2019.  Let's now narrow our data by eliminating columns with greater than 70% missing values.

In [7]:
potholes.dropna(thresh=(.7*potholes.shape[0]), axis=1, inplace=True)
potholes.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,cross_street_1,cross_street_2,...,community_board,borough,x_coordinate_state_plane,y_coordinate_state_plane,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location
0,42035216,2019-03-23 15:37:36,2019-03-27 07:34:00,DOT,Department of Transportation,Street Condition,Pothole,11355.0,146 STREET,148 STREET,...,07 QUEENS,QUEENS,1033984.0,210706.0,UNKNOWN,Unspecified,QUEENS,40.744876,-73.820516,POINT (-73.820515773594 40.744876037242)
1,42035218,2019-03-23 17:09:03,2019-03-27 10:13:00,DOT,Department of Transportation,Street Condition,Pothole,10314.0,NaN,NaN,...,01 STATEN ISLAND,STATEN ISLAND,950223.0,162661.0,UNKNOWN,Unspecified,STATEN ISLAND,40.613078,-74.122557,POINT (-74.122556992381 40.613077936948)
2,42036611,2019-03-23 19:04:12,NaT,DOT,Department of Transportation,Street Condition,Pothole,11377.0,NaN,NaN,...,02 QUEENS,QUEENS,1012316.0,209017.0,UNKNOWN,Unspecified,QUEENS,40.740335,-73.898720,POINT (-73.898720018464 40.740335470005)
3,42036612,2019-03-23 09:25:59,2019-03-27 13:20:00,DOT,Department of Transportation,Street Condition,Pothole,10314.0,NaN,NaN,...,01 STATEN ISLAND,STATEN ISLAND,945579.0,163320.0,UNKNOWN,Unspecified,STATEN ISLAND,40.614868,-74.139287,POINT (-74.139287300499 40.614867702584)
4,42036613,2019-03-23 13:40:15,2019-03-28 09:44:00,DOT,Department of Transportation,Street Condition,Pothole,10475.0,CARVER LOOP,GIVAN AVENUE,...,10 BRONX,BRONX,1031923.0,259965.0,UNKNOWN,Unspecified,BRONX,40.880089,-73.827604,POINT (-73.827603802742 40.880089328815)


Let's add a new column that gives the time between complaint creation date and completion date (either closed or resolution updated date), and remove the columns we don't need any more:

In [8]:
import numpy as np
potholes['resolved_date'] = np.where(potholes['closed_date'].notnull(), potholes['closed_date'], 
                                     potholes['resolution_action_updated_date'])
potholes['days_to_close'] = (potholes['resolved_date'].dt.date - potholes['created_date'].dt.date).dt.days
potholes.drop(columns=['closed_date','resolution_action_updated_date'], inplace = True)

We can also remove columns that don't provide meaningful data for prediction (like `unique_key`) or have the same data throughout (like `agency`).  Let's take a quick peek at the number of unique values in each column to see if there are obvious candidates for removal:

Let's take another peek at our column information:



In [9]:
potholes.nunique()

unique_key                  569142
created_date                545013
agency                           1
agency_name                      1
complaint_type                   2
descriptor                       1
incident_zip                   229
cross_street_1                7836
cross_street_2                7616
address_type                     3
city                            89
status                           5
resolution_description          32
community_board                 77
borough                          6
x_coordinate_state_plane     93743
y_coordinate_state_plane    100154
open_data_channel_type           4
park_facility_name               1
park_borough                     6
latitude                    216460
longitude                   216458
location                    216460
resolved_date               409077
days_to_close                  286
dtype: int64

OK, so we can get rid of `agency`, `agency_name`, `descriptor`, and `park_facility_name` for sure!  As stated earlier, `unique_key` doesn't add any useful info, so we can get of it as well.  Location is essentially a duplicate of lat/long, so we can get rid of that column, too.  Unfortunately, we don't have any information about `x_ccordinate_state_plane` and `y_coordinate_state_plane`, so we'll remove them.  There's not much we can do with that data!

In [10]:
potholes.drop(columns = ['unique_key', 'agency', 'agency_name', 'descriptor', 
                         'park_facility_name', 'location', 'x_coordinate_state_plane', 
                         'y_coordinate_state_plane'], inplace = True)

Let's peek at the column data now:

In [11]:
potholes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569142 entries, 0 to 569141
Data columns (total 17 columns):
created_date              569142 non-null datetime64[ns]
complaint_type            569142 non-null object
incident_zip              525046 non-null float64
cross_street_1            462612 non-null object
cross_street_2            462547 non-null object
address_type              551495 non-null object
city                      528465 non-null object
status                    569142 non-null object
resolution_description    568606 non-null object
community_board           569142 non-null object
borough                   569142 non-null object
open_data_channel_type    569142 non-null object
park_borough              569142 non-null object
latitude                  521193 non-null float64
longitude                 521193 non-null float64
resolved_date             568949 non-null datetime64[ns]
days_to_close             568949 non-null float64
dtypes: datetime64[ns](2), float64(4

Great, that's  a compact DataFrame with not very many missing values and not a lot of duplication of data across columns.  Let's save that data!

In [12]:
potholes.to_csv("../data/cleaned_311_pothole_data.csv", index=False)